<a href="https://colab.research.google.com/github/vivekkishore/machinelearning/blob/main/querygpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###import library and key

In [2]:
!pip install python-dotenv
!pip install openai
!pip install --upgrade langchain
!pip install tiktoken
!pip install chromadb
import warnings
warnings.filterwarnings('ignore')
import langchain
langchain.debug=False
import os, openai
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())
openai.api_key=os.environ['API_KEY']


###Load and split

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter

#Load
loader=TextLoader('/content/sample1.txt')
docs=loader.load()

#Split
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10,separators=["\n\n", "\n", " ", ""])
texts=text_splitter.split_documents(docs)



###LLM embedding and storing in VectorDB

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


from langchain.vectorstores import Chroma
persist_directory = '/content/chroma/'

# to remove old database files
# !rm -rf ./content/chroma/

db=Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)


###retrieval from VectorDB/ Through LLM

In [30]:
# #fetching relevant data directly from vector db using similarilty search
# Query='Is entity id required for all kinds of applications?'
# # result=db.similarity_search(Query, k=2)
# result1=db.max_marginal_relevance_search(Query,fetch_k=4,k=3)

from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0,openai_api_key= openai.api_key)

# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

# Run chain
from langchain.chains import RetrievalQA

def run_model(question):
  qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type='mmr',search_kwargs={'k': 3, 'fetch_k':4 }),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, verbose =True)
  result=qa_chain({"query":question})
  return result


In [ ]:
run_model("what is step 3 in SAML integration process?")

###Memory store inclusion in the flow

In [48]:
# including memory to store past conversation
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
from langchain.chains import ConversationalRetrievalChain

def run_memory_chain(question):
  qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=db.as_retriever(search_type='similarity',search_kwargs={'k': 3, 'fetch_k':4 }),
    chain_type="stuff",
    memory=memory,
    verbose=True)

  result =qa({"question":question})
  return result



In [52]:
run_memory_chain("what is step 3 in SAML integration process?")


# question = "why is he like this?"
# result = qa({"question": question})
# result

# question = "shall we all be like him?"
# result = qa({"question": question})
# result

# memory.clear()



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: what is step 3 in SAML integration process?
Assistant: Step 3 in the SAML integration process is to manage the SAML signing certificate. This involves using a certificate to sign the SAML tokens that Azure AD sends to the application. You can view or download the active certificate with the help of the ESO team. If there is a need to update, create, or import a certificate, you should fill out relevant RITMs with the ESO team.
Follow Up Input: what is step 3 in SAML integration process?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know

{'question': 'what is step 3 in SAML integration process?',
 'chat_history': [HumanMessage(content='what is step 3 in SAML integration process?', additional_kwargs={}, example=False),
  AIMessage(content='Step 3 in the SAML integration process is to manage the SAML signing certificate. This involves using a certificate to sign the SAML tokens that Azure AD sends to the application. You can view or download the active certificate with the help of the ESO team. If there is a need to update, create, or import a certificate, you should fill out relevant RITMs with the ESO team.', additional_kwargs={}, example=False),
  HumanMessage(content='what is step 3 in SAML integration process?', additional_kwargs={}, example=False),
  AIMessage(content='The purpose of managing the SAML signing certificate in step 3 is to ensure that Azure AD can sign the SAML tokens it sends to the application. The certificate is used to verify the authenticity and integrity of the tokens. If there is a need to upda

In [51]:
# memory.clear()
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='what is step 3 in SAML integration process?', additional_kwargs={}, example=False), AIMessage(content='Step 3 in the SAML integration process is to manage the SAML signing certificate. This involves using a certificate to sign the SAML tokens that Azure AD sends to the application. You can view or download the active certificate with the help of the ESO team. If there is a need to update, create, or import a certificate, you should fill out relevant RITMs with the ESO team.', additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history')